In [51]:
%matplotlib inline
from collections import Counter

# members = pd.read_csv("data/members_v3.csv")
# train = pd.read_csv("data/train_v2.csv")
# transactions = pd.read_csv("data/transactions_v2.csv")
# user_logs = pd.read_csv("data/user_logs_v2.csv")
# sample_submission = pd.read_csv("data/sample_submission_v2.csv")
train = pd.read_csv("data/train_final.csv")
test = pd.read_csv("data/test_final.csv")

In [52]:
train.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,180.0,0.0,20170311.0,20170411.0,0.0,...,2.000000,0.000000,26.000000,30.00000,6986.509000,13.0,male,3.0,20131223.0,20.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,300.0,0.0,20170314.0,20170615.0,0.0,...,32.000000,22.000000,205.000000,432.00000,67810.467000,13.0,male,3.0,20131227.0,18.0
2,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,20151208.0,20180108.0,0.0,...,15.659063,17.442416,483.033693,472.74419,126291.230177,1.0,female,7.0,20140109.0,38.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,99.0,1.0,20170316.0,20170918.0,0.0,...,4.000000,4.000000,962.000000,548.00000,239882.241000,13.0,female,7.0,20140125.0,35.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,40.0,30.0,149.0,149.0,1.0,20170316.0,20170314.0,1.0,...,1.000000,4.000000,37.000000,48.00000,10110.665000,22.0,female,3.0,20140126.0,26.0


In [54]:
train.shape, test.shape

((933578, 22), (907471, 22))

In [55]:
train.dtypes

msno                       object
is_churn                    int64
payment_method_id         float64
payment_plan_days         float64
plan_list_price           float64
actual_amount_paid        float64
is_auto_renew             float64
transaction_date          float64
membership_expire_date    float64
is_cancel                 float64
num_25                    float64
num_50                    float64
num_75                    float64
num_985                   float64
num_100                   float64
num_unq                   float64
total_secs                float64
city                      float64
gender                     object
registered_via            float64
registration_init_time    float64
age                       float64
dtype: object

In [56]:
train['is_churn'] = train['is_churn'].astype('category')
train['payment_method_id'] = train['payment_method_id'].astype('category')
train['payment_plan_days'] = train['payment_plan_days'].astype('category')
train['is_auto_renew'] = train['is_auto_renew'].astype('category')
train['transaction_date'] = train['transaction_date'].astype('int')
train['membership_expire_date'] = train['membership_expire_date'].astype('int')
train['is_cancel'] = train['is_cancel'].astype('category')
train['city'] = train['city'].astype('category')
train['gender'] = train['gender'].astype('category')
train['registered_via'] = train['registered_via'].astype('category')
train['registration_init_time'] = train['registration_init_time'].astype('int')
train['age'] = train['age'].astype('int')

In [57]:
train['transaction_date'] = pd.to_datetime(train['transaction_date'], format='%Y%m%d')
train['membership_expire_date'] = pd.to_datetime(train['membership_expire_date'], format='%Y%m%d')

In [62]:
train[train['registration_init_time'] < 2000] # registration_init_time 이상값 108210개

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age
20,NTOh3fAZrLoIyE1lxoRpp2lzEraNnnWbUvOhrmO1bUM=,1,41.0,30.0,149.0,149.0,1.0,2015-12-19,2017-05-20,0.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,13.0,female,3.0,26,50
39,+wisdxP166qHezQOKLPv4l4AfZClRF3ExWGD2rzYYCU=,1,41.0,30.0,149.0,149.0,1.0,2017-03-06,2017-03-06,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,male,9.0,26,27
49,P9FjH43/pjIS6PwcXP8VcA+P4RTq/mH+F3DLuXDpFkE=,1,41.0,30.0,99.0,99.0,1.0,2017-03-22,2017-03-22,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,4.0,female,9.0,29,23
53,Ybp7mbACy/qnytzEr8rZykiEyL23ESsAIZRo17VAT/8=,1,41.0,30.0,99.0,99.0,1.0,2017-03-11,2017-03-11,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,13.0,female,7.0,34,40
78,r0ND2OhGFAUltb62mwbwM7W9nnI+MMA3IAdBHk3abbI=,1,41.0,30.0,99.0,99.0,1.0,2017-03-20,2017-03-20,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,13.0,female,4.0,28,19
88,YOF2Z5cnRQQzJ0+jO1k+0nkQ5rO2KDZA1x+XbZ2vPaw=,1,41.0,30.0,99.0,99.0,1.0,2017-03-07,2017-03-07,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,6.0,male,9.0,29,52
89,e0N3u3laGuzrocxSIezxFVU+uNuiiN4RAAhr36rmV0Y=,1,41.0,30.0,99.0,99.0,1.0,2017-03-10,2017-03-10,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,male,7.0,27,28
96,47WmILK1vHIz0iCDMX1elQNorVhh1aupLKq1R5ddBto=,1,41.0,30.0,99.0,99.0,1.0,2017-03-12,2017-03-12,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,15.0,female,9.0,25,29
117,8Yfas8KHQ5UHsRwQ3Iesv2UXZ9FStQxCZ+TKaJ7Q+SA=,1,30.0,30.0,129.0,129.0,1.0,2017-03-03,2017-03-03,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,female,9.0,19,45
127,ZpboqV+244iMAeyKObEcf057ZQlDI5ndU9aqq6dABKc=,1,41.0,30.0,149.0,149.0,1.0,2017-03-07,2017-03-07,1.0,...,15.659063,17.442416,483.033693,472.744190,126291.230177,1.0,female,4.0,37,29


In [63]:
a = train['plan_list_price'] - train['actual_amount_paid']
count = 0
for i in a:
    if i != 0:
        count += 1
        
count # discount를 받은 것으로 추정

1712

In [64]:
train['disc_user'] = a

ls = []
for j in train['disc_user'][train['disc_user'] > 0].index:
    ls.append(j)

train.set_value(index=ls, col='disc_user', value=1)
train['disc_user'] = train['disc_user'].astype('category')
train.drop(labels='plan_list_price', axis=1, inplace=True)

In [69]:
train.head()

,msno,is_churn,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,num_25,...,num_985,num_100,num_unq,total_secs,city,gender,registered_via,registration_init_time,age,disc_user
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,0.0,2017-03-11,2017-04-11,0.0,0.000000,...,0.000000,26.000000,30.00000,6986.509000,13.0,male,3.0,20131223,20,0.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,0.0,2017-03-14,2017-06-15,0.0,239.000000,...,22.000000,205.000000,432.00000,67810.467000,13.0,male,3.0,20131227,18,0.0
2,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,1.0,2015-12-08,2018-01-08,0.0,103.390612,...,17.442416,483.033693,472.74419,126291.230177,1.0,female,7.0,20140109,38,0.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,1.0,2017-03-16,2017-09-18,0.0,9.000000,...,4.000000,962.000000,548.00000,239882.241000,13.0,female,7.0,20140125,35,0.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,40.0,30.0,149.0,1.0,2017-03-16,2017-03-14,1.0,6.000000,...,4.000000,37.000000,48.00000,10110.665000,22.0,female,3.0,20140126,26,0.0


In [70]:
train.dtypes

msno                              object
is_churn                        category
payment_method_id               category
payment_plan_days               category
actual_amount_paid               float64
is_auto_renew                   category
transaction_date          datetime64[ns]
membership_expire_date    datetime64[ns]
is_cancel                       category
num_25                           float64
num_50                           float64
num_75                           float64
num_985                          float64
num_100                          float64
num_unq                          float64
total_secs                       float64
city                            category
gender                          category
registered_via                  category
registration_init_time             int32
age                                int32
disc_user                       category
dtype: object

In [71]:
train.to_csv("train_final3.csv", index=False)